# Tagging and Extraction Using OpenAI functions

In [32]:
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting

import os
from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) 
_ = load_dotenv(dotenv_path="/Users/maxzhao/.env") 
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]
os.environ['USER_AGENT'] = 'myagent'
AWS_ACCESS_KEY_ID=os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRECT_ACCESS_KEY=os.environ["AWS_SECRECT_ACCESS_KEY"]

In [33]:
#os.environ.clear()

In [34]:
import boto3
import json

# Initialize a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
    region_name='us-east-1'  # e.g., 'us-west-1'
)

# Create an S3 client
s3 = session.client('s3')

# Your JSON object
data = {
    "name": "John Doe",
    "age": 30,
    "city": "New York"
}

# Convert JSON object to string
json_data = json.dumps(data)

# Define your bucket name and the desired object name (key)
bucket_name = 'gz5427'
object_name = 'llm_extract/test.json'

# Upload JSON string to S3
s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)

print(f"Uploaded {object_name} to {bucket_name}")

Uploaded llm_extract/test.json to gz5427


In [35]:
!pip freeze | grep langchain

!pip freeze | grep openai

!pip freeze | grep pydantic


langchain==0.2.15
langchain-community==0.2.13
langchain-core==0.2.41
langchain-openai==0.1.23
langchain-text-splitters==0.2.4
langchain-openai==0.1.23
openai==1.47.0
openapi-schema-pydantic==1.2.4
pydantic==2.9.2
pydantic-settings==2.5.2
pydantic_core==2.23.4


## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [36]:
from langchain_openai import ChatOpenAI
from typing import List, Optional
from pydantic import BaseModel, Field
#from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(temperature=0,model="gpt-4o")

In [37]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [38]:
doc = documents[0]

In [39]:
page_content = doc.page_content[:10000]

Now let's try to extract all papers mentioned in this article

In [40]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [41]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [42]:
paper_extraction_function = [convert_to_openai_function(Info)]
extraction_model = model.bind(functions=paper_extraction_function, function_call={"name":"Info"})

In [43]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [44]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT)', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH)', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD)', 'author': 'Laskin et al. 2023'}]

In [45]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [46]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [47]:
output=extraction_chain.invoke({"input": page_content})
output

[{'title': 'Chain of thought (CoT)', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH)', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD)', 'author': 'Laskin et al. 2023'}]

## Store data into s3.


In [48]:
import boto3
import json

# Initialize a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
    region_name='us-east-1'  # e.g., 'us-west-1'
)

# Create an S3 client
s3 = session.client('s3')

# Convert JSON object to string
json_data = json.dumps(output)

# Define your bucket name and the desired object name (key)
bucket_name = 'gz5427'
object_name = 'llm_extract/extracted_Fromweb.json'

# Upload JSON string to S3
s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)

print(f"Uploaded {object_name} to {bucket_name}")

Uploaded llm_extract/extracted_Fromweb.json to gz5427


In [49]:
####. Method 1: Use this function to store data into S3.
####.           This is probabaly the easiest way and easy to debug.
def storeintoS3(data, bucketname, filepath):
    import boto3
    import json

    # Initialize a session using your AWS credentials
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
        region_name='us-east-1'  # e.g., 'us-west-1'
    )

    # Create an S3 client
    s3 = session.client('s3')
    
    # Convert JSON object to string
    json_data = json.dumps(data)

    # Upload JSON string to S3
    s3.put_object(Bucket=bucketname, Key=filepath, Body=json_data)
  
#####.  Finally store it into S3
storeintoS3(data=output, bucketname='gz5427', filepath='llm_extract/testlcel_v3.json')   